In [52]:
######################
# Process MIDI Files #
######################

from music21 import converter
from music21 import instrument
from music21 import chord
from music21 import note
import glob
import json
import os

# VARS
DATA_GLOB = "data/pokemon/PkmRB/PkmRB-Battle1.mid"
MODEL_DIR = "model/pokemon/PkmRB-V1"
NOTES_FILE = "%s/%s" % (MODEL_DIR, "notes.json")

NOTES_BY_FILE = []
NOTES = set()

for input_file in glob.glob(DATA_GLOB):
    print("Ingesting... %s" % input_file)
    
    midi = converter.parse(input_file)
    parts = instrument.partitionByInstrument(midi)
    if parts:
        notes_to_parse = parts.parts[0].recurse()
    else:
        notes_to_parse = midi.flat.notes
    
    combined_notes = []
    file_notes = []
    last_offset = 0.0
    
    note_num = 1
    for element in notes_to_parse:
        print(type(element))
        
        if isinstance(element, note.Note):
            print("%02d | %02.4f | %s | %s" % (note_num, element.offset, element.duration.type, element.fullName))
            note_num += 1
            
        if (note_num > 200):
            break

    NOTES_BY_FILE.append(file_notes)
    NOTES.update(set(file_notes))

NOTES = sorted(NOTES)

if not os.path.exists(os.path.dirname(NOTES_FILE)):
    os.makedirs(os.path.dirname(NOTES_FILE))
with open(NOTES_FILE, 'w') as note_file:
    json.dump(NOTES, note_file)
    


Ingesting... data/pokemon/PkmRB/PkmRB-Battle1.mid
<class 'music21.stream.Part'>
<class 'music21.instrument.Instrument'>
<class 'music21.tempo.MetronomeMark'>
<class 'music21.key.Key'>
<class 'music21.key.Key'>
<class 'music21.key.Key'>
<class 'music21.meter.TimeSignature'>
<class 'music21.meter.TimeSignature'>
<class 'music21.meter.TimeSignature'>
<class 'music21.note.Note'>
01 | 0.0000 | 16th | C in octave 5 16th Note
<class 'music21.note.Note'>
02 | 0.0000 | 16th | G in octave 5 16th Note
<class 'music21.note.Note'>
03 | 0.0000 | 16th | C-sharp in octave 4 16th Note
<class 'music21.note.Note'>
04 | 0.2500 | 16th | B in octave 4 16th Note
<class 'music21.note.Note'>
05 | 0.2500 | 16th | F-sharp in octave 5 16th Note
<class 'music21.note.Rest'>
<class 'music21.note.Note'>
06 | 0.5000 | 16th | B-flat in octave 4 16th Note
<class 'music21.note.Note'>
07 | 0.5000 | 16th | F in octave 5 16th Note
<class 'music21.note.Note'>
08 | 0.5000 | 16th | C-sharp in octave 4 16th Note
<class 'music21

In [5]:
import uuid
from music21 import note, chord, stream

MIDI_OUTPUT_DIRECTORY = "experimental/pokemon/PkmRB-V1/"

output = NOTES_BY_FILE[0]
    
offset = 0
output_notes = []
for note_pattern in output:
    if ('.' in note_pattern) or note_pattern.isdigit():
        notes_in_chord = note_pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note = storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    else:
        new_note = note.Note(note_pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
        
    offset += 0.5
    
if not os.path.exists(os.path.dirname(MIDI_OUTPUT_DIRECTORY)):
    os.makedirs(os.path.dirname(MIDI_OUTPUT_DIRECTORY))
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp="%s/%s.%s.%s" % (MIDI_OUTPUT_DIRECTORY, "exp1", uuid.uuid4(), "mid"))

'experimental/pokemon/PkmRB-V1//exp1.66f3faa7-2010-4fb9-adb2-5b6a48f8cee1.mid'

In [8]:
########################
# Build Note Sequences #
########################

import numpy
from keras.utils import np_utils

NOTES_FILE = None
SEQUENCE_LENGTH = 80
X = []
y = []

# Load Notes from disk if desired
if NOTES_FILE:
    with open(NOTES_FILE, 'r') as notes_file:
        NOTES = json.loads(notes_file.read())

NOTE_TO_INT = dict((note, number) for number, note in enumerate(NOTES))
INT_TO_NOTE = dict((number, note) for number, note in enumerate(NOTES))

for notes in NOTES_BY_FILE:
    for i in range(0, len(notes) - SEQUENCE_LENGTH, 1):
        input_sequence = notes[i:i + SEQUENCE_LENGTH] # N Input Notes
        output_sequence = notes[i + SEQUENCE_LENGTH] # 1 Output Note
        
        X.append([NOTE_TO_INT[c] for c in input_sequence])
        y.append(NOTE_TO_INT[output_sequence])
        
N_SEQUENCES = len(X)
N_VOCAB = len(NOTES)

print("Num Sequences: %d" % N_SEQUENCES)
print("Num Vocab: %d" % N_VOCAB)

X = numpy.reshape(X, (N_SEQUENCES, SEQUENCE_LENGTH, 1)) / N_VOCAB
y = np_utils.to_categorical(y)

Num Sequences: 25524
Num Vocab: 56


In [9]:
#######################
# Construct the Model #
#######################

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.callbacks import ModelCheckpoint

MODEL_FILE = "%s/%s" % (MODEL_DIR, "LSTM-L256-L128-S80-weights-improvements-013-2.9983.hdf5")
CHECKPOINT_FILE = MODEL_FILE

model = Sequential()
model.add(LSTM(
    256,
    input_shape=(X.shape[1], X.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(128))
model.add(Dropout(0.3))
model.add(Dense(N_VOCAB))
model.add(Activation('softmax'))
if (CHECKPOINT_FILE):
    model.load_weights(CHECKPOINT_FILE)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
##################
# Produce a MIDI #
##################

import uuid
from music21 import note, chord, stream

MIDI_OUTPUT_DIRECTORY = "experimental/pokemon/PkmRB-V1/"

SEED_INPUT_SEQUENCE = X[numpy.random.randint(0, len(X))] * N_VOCAB
SEED_SEQUENCE = [INT_TO_NOTE[numpy.round(number[0])] for number in SEED_INPUT_SEQUENCE]

for i in range(500):
    input_sequence = numpy.reshape(pattern, (1, len(pattern), 1))
    output_sequence = model.predict(input_sequence, verbose=0)
    
    number = numpy.argmax(output_sequence)
    result = INT_TO_NOTE[number]
    output.append(result)
    
    pattern = numpy.append(pattern, number)
    pattern = pattern[1:]
    
offset = 0
output_notes = []
for note_pattern in output:
    if ('.' in note_pattern) or note_pattern.isdigit():
        notes_in_chord = note_pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note = storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    else:
        new_note = note.Note(note_pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
        
    offset += 0.5
    
if not os.path.exists(os.path.dirname(MIDI_OUTPUT_DIRECTORY)):
    os.makedirs(os.path.dirname(MIDI_OUTPUT_DIRECTORY))
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp="%s/%s.%s.%s" % (MIDI_OUTPUT_DIRECTORY, os.path.basename(CHECKPOINT_FILE), uuid.uuid4(), "mid"))

Epoch 1/500
25524/25524 [==============================] - 30s 1ms/step - loss: 3.5223
Epoch 2/500
25524/25524 [==============================] - 29s 1ms/step - loss: 3.4550
Epoch 3/500
25524/25524 [==============================] - 29s 1ms/step - loss: 3.3991
Epoch 4/500
25524/25524 [==============================] - 29s 1ms/step - loss: 3.3360
Epoch 5/500
16768/25524 [==================>...........] - ETA: 9s - loss: 3.2961 